# Database Schema Analysis & Entity Extraction Optimization

This notebook analyzes the actual Supabase database schema and optimizes the entity extraction system to achieve 100% test success rate with real data examples including mouse, USB, keyboard products and proper date filtering.

## Goals:
1. **Fix database schema mismatches** - Align entity field mappings with actual Supabase tables
2. **Optimize temporal filtering** - Ensure proper date range filtering works correctly  
3. **Enhance product detection** - Add real product data like mouse, USB, keyboard, monitor
4. **Achieve 100% test success** - Fix all failing test cases to get actual results
5. **Validate hover functionality** - Ensure temporal and pronoun hover text works perfectly

Let's start by analyzing the current database structure and identifying issues.

In [ ]:
// Import Required Libraries and Services
import { SupabaseService } from './src/services/supabase.js';
import { ENTITY_FIELDS_MAP } from './src/services/entityFieldsMap.js';

// Initialize Supabase service for schema analysis
const supabaseService = new SupabaseService();
const client = supabaseService.getClient();

console.log('🔍 Database Schema Analysis Starting...');
console.log('📊 Current date for temporal testing:', new Date().toLocaleDateString());

In [ ]:
// Step 1: Inspect Actual Database Schema
async function inspectDatabaseSchema() {
  const tables = ['customers', 'products', 'sales', 'tasks', 'users', 'stock', 'shifts', 'attendance'];
  const schemaAnalysis: { [key: string]: any } = {};
  
  console.log('🔍 ANALYZING ACTUAL DATABASE SCHEMA...\n');
  
  for (const tableName of tables) {
    try {
      // Get actual column information from PostgreSQL
      const { data: columns, error } = await client
        .from('information_schema.columns')
        .select('column_name, data_type, is_nullable')
        .eq('table_name', tableName)
        .order('ordinal_position');
      
      if (columns && columns.length > 0) {
        schemaAnalysis[tableName] = {
          actualColumns: columns,
          mappedFields: ENTITY_FIELDS_MAP[tableName]?.fields || [],
        };
        
        console.log(`📋 Table: ${tableName.toUpperCase()}`);
        console.log('   Actual Columns:', columns.map(c => `${c.column_name} (${c.data_type})`).join(', '));
        
        // Check for mismatches
        const mappedFieldNames = ENTITY_FIELDS_MAP[tableName]?.fields.map(f => f.name) || [];
        const actualColumnNames = columns.map(c => c.column_name);
        
        const missingInActual = mappedFieldNames.filter(f => !actualColumnNames.includes(f));
        const missingInMapping = actualColumnNames.filter(c => !mappedFieldNames.includes(c));
        
        if (missingInActual.length > 0) {
          console.log(`   ❌ Missing in actual DB:`, missingInActual);
        }
        if (missingInMapping.length > 0) {
          console.log(`   ⚠️ Missing in mapping:`, missingInMapping);
        }
        if (missingInActual.length === 0 && missingInMapping.length === 0) {
          console.log(`   ✅ Schema mapping is correct`);
        }
      } else {
        console.log(`❌ Table ${tableName} not found or inaccessible`);
      }
      
      console.log('');
    } catch (error) {
      console.error(`Error inspecting ${tableName}:`, error);
    }
  }
  
  return schemaAnalysis;
}

// Run schema inspection
const schemaAnalysis = await inspectDatabaseSchema();

In [ ]:
// Step 2: Analyze Real Product Data for Better Entity Detection
async function analyzeProductData() {
  console.log('🛍️ ANALYZING REAL PRODUCT DATA...\n');
  
  try {
    // Get sample products to understand naming patterns
    const { data: products, error } = await client
      .from('products')
      .select('name, description, category, sku')
      .limit(10);
    
    if (products && products.length > 0) {
      console.log('📦 SAMPLE PRODUCTS:');
      
      const productTerms = new Set<string>();
      const categoryTerms = new Set<string>();
      
      products.forEach((product, index) => {
        console.log(`${index + 1}. ${product.name} (${product.category})`);
        console.log(`   Description: ${product.description}`);
        console.log(`   SKU: ${product.sku}`);
        
        // Extract product terms for entity detection
        const nameWords = product.name.toLowerCase().split(/\s+/);
        const descWords = product.description?.toLowerCase().split(/\s+/) || [];
        
        nameWords.forEach(word => {
          if (word.length > 2 && /^[a-z]+$/.test(word)) {
            productTerms.add(word);
          }
        });
        
        if (product.category) {
          categoryTerms.add(product.category.toLowerCase());
        }
        
        console.log('');
      });
      
      console.log('🏷️ DETECTED PRODUCT TERMS:');
      console.log('Single words:', Array.from(productTerms).slice(0, 20).join(', '));
      console.log('Categories:', Array.from(categoryTerms).join(', '));
      
      // Look for technology products specifically
      const techProducts = products.filter(p => 
        p.name.toLowerCase().includes('mouse') ||
        p.name.toLowerCase().includes('keyboard') ||
        p.name.toLowerCase().includes('monitor') ||
        p.name.toLowerCase().includes('laptop') ||
        p.name.toLowerCase().includes('usb') ||
        p.name.toLowerCase().includes('cable')
      );
      
      console.log(`\n💻 TECHNOLOGY PRODUCTS FOUND: ${techProducts.length}`);
      techProducts.forEach(product => {
        console.log(`   • ${product.name} - ${product.category}`);
      });
      
      return { products, productTerms: Array.from(productTerms), categoryTerms: Array.from(categoryTerms) };
    } else {
      console.log('❌ No products found in database');
      return { products: [], productTerms: [], categoryTerms: [] };
    }
  } catch (error) {
    console.error('Error analyzing product data:', error);
    return { products: [], productTerms: [], categoryTerms: [] };
  }
}

// Run product analysis
const productAnalysis = await analyzeProductData();

In [ ]:
// Step 3: Analyze Temporal Data and Fix Date Filtering
async function analyzeTemporalData() {
  console.log('📅 ANALYZING TEMPORAL DATA AND DATE FIELDS...\n');
  
  const tables = ['sales', 'tasks', 'customers', 'products', 'users', 'shifts', 'attendance'];
  const temporalAnalysis: { [key: string]: any } = {};
  
  for (const tableName of tables) {
    try {
      // Get date fields for this table
      const { data: dateColumns, error } = await client
        .from('information_schema.columns')
        .select('column_name, data_type')
        .eq('table_name', tableName)
        .in('data_type', ['date', 'timestamp without time zone', 'timestamp with time zone', 'time without time zone']);
      
      if (dateColumns && dateColumns.length > 0) {
        console.log(`📋 ${tableName.toUpperCase()} - Date Fields:`);
        dateColumns.forEach(col => {
          console.log(`   • ${col.column_name} (${col.data_type})`);
        });
        
        // Get sample data with date ranges
        const { data: sampleData, error: dataError } = await client
          .from(tableName)
          .select('*')
          .limit(3);
        
        if (sampleData && sampleData.length > 0) {
          console.log(`   Sample dates from ${tableName}:`);
          sampleData.forEach((record, i) => {
            const dateFields = dateColumns.map(col => col.column_name);
            const dateValues = dateFields.map(field => {
              const value = record[field];
              return value ? `${field}: ${new Date(value).toLocaleDateString()}` : `${field}: null`;
            }).join(', ');
            console.log(`   ${i + 1}. ${dateValues}`);
          });
        }
        
        temporalAnalysis[tableName] = {
          dateFields: dateColumns.map(c => c.column_name),
          dataTypes: dateColumns.map(c => c.data_type),
          sampleData: sampleData?.slice(0, 2) || []
        };
      } else {
        console.log(`📋 ${tableName.toUpperCase()} - No date fields found`);
        temporalAnalysis[tableName] = { dateFields: [], dataTypes: [], sampleData: [] };
      }
      
      console.log('');
    } catch (error) {
      console.error(`Error analyzing temporal data for ${tableName}:`, error);
    }
  }
  
  // Test temporal filtering with real dates
  console.log('🧪 TESTING TEMPORAL FILTERING...\n');
  
  try {
    // Test with sales table
    const today = new Date().toISOString().split('T')[0];
    const { data: todaySales, error } = await client
      .from('sales')
      .select('*')
      .gte('created_at', today)
      .lte('created_at', today + 'T23:59:59');
    
    console.log(`Sales from today (${today}): ${todaySales?.length || 0} records`);
    
    // Test with tasks due dates
    const { data: todayTasks } = await client
      .from('tasks')
      .select('*')
      .gte('due_date', today)
      .lte('due_date', today + 'T23:59:59');
    
    console.log(`Tasks due today: ${todayTasks?.length || 0} records`);
    
  } catch (error) {
    console.error('Error testing temporal filtering:', error);
  }
  
  return temporalAnalysis;
}

// Run temporal analysis
const temporalAnalysis = await analyzeTemporalData();

In [ ]:
// Step 4: Generate Optimization Recommendations
function generateOptimizationRecommendations(schemaAnalysis: any, productAnalysis: any, temporalAnalysis: any) {
  console.log('🔧 GENERATING OPTIMIZATION RECOMMENDATIONS...\n');
  
  const recommendations = {
    schemaFixes: [] as string[],
    entityDetectionImprovements: [] as string[],
    temporalFilteringFixes: [] as string[],
    productDataEnhancements: [] as string[],
    testingImprovements: [] as string[]
  };
  
  // Schema fixes
  Object.entries(schemaAnalysis).forEach(([tableName, analysis]: [string, any]) => {
    if (analysis.actualColumns) {
      const mappedFieldNames = ENTITY_FIELDS_MAP[tableName]?.fields.map(f => f.name) || [];
      const actualColumnNames = analysis.actualColumns.map((c: any) => c.column_name);
      
      const missingInActual = mappedFieldNames.filter(f => !actualColumnNames.includes(f));
      const missingInMapping = actualColumnNames.filter(c => !mappedFieldNames.includes(c));
      
      if (missingInActual.length > 0) {
        recommendations.schemaFixes.push(`Fix ${tableName}: Remove non-existent fields ${missingInActual.join(', ')} from entity mapping`);
      }
      if (missingInMapping.length > 0) {
        recommendations.schemaFixes.push(`Enhance ${tableName}: Add missing fields ${missingInMapping.join(', ')} to entity mapping`);
      }
    }
  });
  
  // Entity detection improvements
  if (productAnalysis.productTerms.length > 0) {
    recommendations.entityDetectionImprovements.push(`Add dynamic product terms: ${productAnalysis.productTerms.slice(0, 10).join(', ')}`);
  }
  if (productAnalysis.categoryTerms.length > 0) {
    recommendations.entityDetectionImprovements.push(`Add category-based detection for: ${productAnalysis.categoryTerms.join(', ')}`);
  }
  
  // Temporal filtering fixes
  Object.entries(temporalAnalysis).forEach(([tableName, analysis]: [string, any]) => {
    if (analysis.dateFields.length > 0) {
      const hasTimestamp = analysis.dataTypes.some((type: string) => type.includes('timestamp'));
      if (hasTimestamp) {
        recommendations.temporalFilteringFixes.push(`Fix ${tableName}: Use timestamp range filtering instead of exact date matching`);
      }
    }
  });
  
  // Product data enhancements
  recommendations.productDataEnhancements.push('Add real product examples: gaming mouse, wireless keyboard, USB-C cable, 4K monitor');
  recommendations.productDataEnhancements.push('Implement fuzzy product matching for typos and variations');
  recommendations.productDataEnhancements.push('Add product alias detection (e.g., "mouse" → "computer mouse", "wireless mouse")');
  
  // Testing improvements
  recommendations.testingImprovements.push('Use actual database dates for temporal test cases');
  recommendations.testingImprovements.push('Test with real product names from database');
  recommendations.testingImprovements.push('Add negative test cases for non-existent data');
  recommendations.testingImprovements.push('Implement data seeding for consistent test results');
  
  // Print recommendations
  console.log('📋 SCHEMA FIXES:');
  recommendations.schemaFixes.forEach((fix, i) => console.log(`   ${i + 1}. ${fix}`));
  
  console.log('\n🔍 ENTITY DETECTION IMPROVEMENTS:');
  recommendations.entityDetectionImprovements.forEach((imp, i) => console.log(`   ${i + 1}. ${imp}`));
  
  console.log('\n📅 TEMPORAL FILTERING FIXES:');
  recommendations.temporalFilteringFixes.forEach((fix, i) => console.log(`   ${i + 1}. ${fix}`));
  
  console.log('\n🛍️ PRODUCT DATA ENHANCEMENTS:');
  recommendations.productDataEnhancements.forEach((enh, i) => console.log(`   ${i + 1}. ${enh}`));
  
  console.log('\n🧪 TESTING IMPROVEMENTS:');
  recommendations.testingImprovements.forEach((imp, i) => console.log(`   ${i + 1}. ${imp}`));
  
  return recommendations;
}

// Generate recommendations
const recommendations = generateOptimizationRecommendations(schemaAnalysis, productAnalysis, temporalAnalysis);

In [ ]:
// Step 5: Implement Enhanced Levenshtein Distance for Fuzzy Entity Matching
function enhancedLevenshteinDistance(str1: string, str2: string): number {
  const matrix = [];
  const len1 = str1.length;
  const len2 = str2.length;
  
  // Initialize matrix
  for (let i = 0; i <= len2; i++) {
    matrix[i] = [i];
  }
  
  for (let j = 0; j <= len1; j++) {
    matrix[0][j] = j;
  }
  
  // Fill matrix with edit distances
  for (let i = 1; i <= len2; i++) {
    for (let j = 1; j <= len1; j++) {
      const char1 = str1.charAt(j - 1).toLowerCase();
      const char2 = str2.charAt(i - 1).toLowerCase();
      
      if (char1 === char2) {
        matrix[i][j] = matrix[i - 1][j - 1];
      } else {
        matrix[i][j] = Math.min(
          matrix[i - 1][j - 1] + 1, // substitution
          matrix[i][j - 1] + 1,     // insertion
          matrix[i - 1][j] + 1      // deletion
        );
      }
    }
  }
  
  return matrix[len2][len1];
}

// Calculate similarity percentage
function calculateSimilarity(str1: string, str2: string): number {
  const distance = enhancedLevenshteinDistance(str1, str2);
  const maxLength = Math.max(str1.length, str2.length);
  
  if (maxLength === 0) return 100;
  
  return Math.round(((maxLength - distance) / maxLength) * 100);
}

// Find best matches for fuzzy entity detection
function findBestMatches(input: string, candidates: string[], threshold: number = 70): Array<{term: string, similarity: number}> {
  return candidates
    .map(candidate => ({
      term: candidate,
      similarity: calculateSimilarity(input.toLowerCase(), candidate.toLowerCase())
    }))
    .filter(match => match.similarity >= threshold)
    .sort((a, b) => b.similarity - a.similarity);
}

// Test fuzzy matching with product terms
console.log('🔍 TESTING FUZZY ENTITY MATCHING...\n');

const testInputs = ['mose', 'keybord', 'moniter', 'compter', 'acme corp'];
const candidates = [
  'mouse', 'keyboard', 'monitor', 'computer', 'laptop', 'tablet', 
  'acme corporation', 'microsoft', 'apple', 'wireless', 'bluetooth'
];

testInputs.forEach(input => {
  console.log(`Input: "${input}"`);
  const matches = findBestMatches(input, candidates, 60);
  if (matches.length > 0) {
    console.log(`   Best matches:`, matches.slice(0, 3).map(m => `${m.term} (${m.similarity}%)`).join(', '));
  } else {
    console.log(`   No good matches found`);
  }
  console.log('');
});

In [ ]:
// Step 6: Generate Fixed Implementation Code
function generateFixedChatService() {
  console.log('🔧 GENERATING FIXED CHAT SERVICE IMPLEMENTATION...\n');
  
  const fixedCode = `
// FIXED: Enhanced temporal filtering with proper timestamp handling
async function buildDynamicQueryFixed(tableName: string, filters: EntityMatch[], userName?: string) {
  const client = supabaseService.getClient();
  let query = client.from(tableName).select('*');
  let hasFilters = false;

  for (const filter of filters) {
    if (filter.type === 'temporal' && filter.actualValue) {
      const targetDate = new Date(filter.actualValue);
      const startOfDay = new Date(targetDate);
      startOfDay.setHours(0, 0, 0, 0);
      const endOfDay = new Date(targetDate);
      endOfDay.setHours(23, 59, 59, 999);
      
      // Use proper timestamp range filtering
      const startISO = startOfDay.toISOString();
      const endISO = endOfDay.toISOString();
      
      if (tableName === 'sales') {
        query = query.gte('sale_date', startISO).lte('sale_date', endISO);
      } else if (tableName === 'tasks') {
        query = query.gte('due_date', startISO).lte('due_date', endISO);
      } else if (tableName === 'shifts') {
        query = query.gte('start_time', startISO).lte('start_time', endISO);
      } else {
        query = query.gte('created_at', startISO).lte('created_at', endISO);
      }
      
      hasFilters = true;
    }
    
    if (filter.type === 'info' && filter.actualValue) {
      // Enhanced product matching with real database terms
      if (tableName === 'products') {
        query = query.or(\`name.ilike.%\${filter.actualValue}%,description.ilike.%\${filter.actualValue}%,category.ilike.%\${filter.actualValue}%\`);
      } else if (tableName === 'sales') {
        // Join with products for better product filtering
        query = client.from('sales').select(\`
          *,
          products!inner(name, description, category)
        \`).filter('products.name', 'ilike', \`%\${filter.actualValue}%\`);
      }
      
      hasFilters = true;
    }
  }
  
  return hasFilters ? query.limit(20) : query.limit(10);
}

// FIXED: Enhanced entity detection with real product terms
const ENHANCED_BUSINESS_INFO_TERMS = {
  // Real product types from database analysis
  ...${JSON.stringify(productAnalysis.productTerms.slice(0, 20).reduce((acc: any, term: string) => {
    acc[term] = [term];
    return acc;
  }, {}), null, 2)},
  
  // Multi-word terms (keep existing)
  'wireless mouse': ['wireless mouse', 'bluetooth mouse'],
  'usb cable': ['usb cable', 'usb cord', 'usb-c cable'],
  'gaming laptop': ['gaming laptop', 'gaming notebook'],
  'external monitor': ['external monitor', 'external display'],
  'john doe': ['john doe'],
  'acme corporation': ['acme corporation', 'acme corp'],
  'high priority': ['high priority', 'urgent priority']
};
`;
  
  console.log('✅ Generated fixed implementation focusing on:');
  console.log('   • Proper timestamp range filtering for dates');
  console.log('   • Enhanced product term detection');
  console.log('   • Real database field mapping');
  console.log('   • Fuzzy matching capabilities');
  
  return fixedCode;
}

// Generate the fixes
const fixedImplementation = generateFixedChatService();

In [ ]:
// Step 7: Create Comprehensive Test Validation Strategy
function createTestValidationStrategy() {
  console.log('🧪 CREATING 100% SUCCESS TEST VALIDATION STRATEGY...\n');
  
  const strategy = {
    dataSeeding: {
      description: 'Seed database with consistent test data',
      actions: [
        'Insert mouse, keyboard, monitor products',
        'Create sales records with recent dates',
        'Add tasks with various due dates',
        'Ensure user "Ahmed Hassan" exists'
      ]
    },
    
    temporalTesting: {
      description: 'Fix temporal filtering for 100% accuracy',
      fixes: [
        'Use timestamp ranges instead of exact date matching',
        'Handle timezone issues properly',
        'Test with actual database dates',
        'Support quarterly and monthly ranges'
      ]
    },
    
    entityDetection: {
      description: 'Enhance entity detection for real data',
      improvements: [
        'Dynamic product term detection from database',
        'Fuzzy matching for typos',
        'Multi-word entity prioritization',
        'Context-aware suggestions'
      ]
    },
    
    hoverEnhancement: {
      description: 'Perfect hover functionality',
      features: [
        'Show original temporal + converted date',
        'Show original pronoun + resolved user',
        'Add confidence scores to hover',
        'Display entity type information'
      ]
    },
    
    expectedResults: {
      description: 'Test cases that should return actual data',
      guaranteedResults: [
        '"my tasks" - should find Ahmed Hassan tasks',
        '"shifts today" - should find current shifts',
        '"products" - should find all products',
        '"sales" - should find sales records'
      ]
    }
  };
  
  console.log('📋 DATA SEEDING STRATEGY:');
  strategy.dataSeeding.actions.forEach((action, i) => console.log(`   ${i + 1}. ${action}`));
  
  console.log('\n📅 TEMPORAL TESTING FIXES:');
  strategy.temporalTesting.fixes.forEach((fix, i) => console.log(`   ${i + 1}. ${fix}`));
  
  console.log('\n🔍 ENTITY DETECTION IMPROVEMENTS:');
  strategy.entityDetection.improvements.forEach((imp, i) => console.log(`   ${i + 1}. ${imp}`));
  
  console.log('\n🎯 HOVER ENHANCEMENT FEATURES:');
  strategy.hoverEnhancement.features.forEach((feat, i) => console.log(`   ${i + 1}. ${feat}`));
  
  console.log('\n✅ GUARANTEED RESULT TEST CASES:');
  strategy.expectedResults.guaranteedResults.forEach((test, i) => console.log(`   ${i + 1}. ${test}`));
  
  return strategy;
}

// Create validation strategy
const validationStrategy = createTestValidationStrategy();

console.log('\n🎯 NEXT STEPS TO ACHIEVE 100% SUCCESS:');
console.log('1. Apply the schema fixes identified in analysis');
console.log('2. Implement enhanced temporal filtering with timestamp ranges');
console.log('3. Add real product terms from database to entity detection');
console.log('4. Fix hover text to show original + converted values');
console.log('5. Seed database with consistent test data');
console.log('6. Run comprehensive tests with real data expectations');
console.log('\n✨ Target: 22/22 tests passing with actual database results!');